<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/status_posts_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Looking at responses to just Status posts



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "status_posts"
MODEL_NAME = "status_posts"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 20
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 2.1MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=07ef382f7f0feec8c362ddedd6419660795a7844c793b3daead74fcbfc29a864
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [0]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [0]:
from google.colab import files 
uploaded = files.upload() 


Saving facebook_congress_posts.csv to facebook_congress_posts.csv


In [0]:
import io 
posts = pd.read_csv(io.BytesIO(uploaded['facebook_congress_posts.csv'])) 


In [0]:
status_posts = posts[posts['post_type']=='status']
status_posts.head()

,op_id,op_gender,post_id,post_text,post_type
8,57265377,M,8,President Trump has just nominated judge Neil Gorsuch to the Supreme Court. His strong academic ...,status
13,57265377,M,13,I'll be joining The Sean Hannity Show from the GOP policy retreat in Philadelphia at 3:30pm CT. ...,status
171,57265377,M,171,"I salute my friend, Ted Cruz on standing strong for conservatism and running a campaign founded ...",status
230,57265377,M,230,Happy to be at #CPAC in Washington D.C. We'll be broadcasting LIVE in 15 minutes! Stay tuned.,status
252,57265377,M,252,"No one fought harder for strict constructionism than Antonin Scalia. As a jurist, he was profoun...",status


In [0]:
status_post_ids = status_posts['post_id'].to_list()

In [0]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
train_df = train_df[train_df['post_id'].isin(status_post_ids)]
dev_df = dev_df[dev_df['post_id'].isin(status_post_ids)]

In [0]:
len(train_df)

1944161

In [0]:
len(dev_df)

390738

In [0]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(768673, 8)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [0]:
train_df.op_gender.value_counts()

M    582285
W    186388
Name: op_gender, dtype: int64

In [0]:
###########################
# Execute this if want to do upsampling.
###########################
# from sklearn.utils import resample

# male = train_df[train_df.op_gender=='M']
# female = train_df[train_df.op_gender=='W']

# female_upsampled = resample(female,
#                           replace=True, # sample with replacement
#                           n_samples=len(male), # match number in majority class
#                           random_state=27) # reproducible results
# upsampled_train = pd.concat([male,female_upsampled])
# print(upsampled_train.op_gender.value_counts())


In [0]:
# replace `train_df` with `upsampled_train` if using upsampling.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

# replace `train_df` with `upsampled_train` if using upsampling.
new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [0]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 00 minutes, 55 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [0]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
| [4 files][  2.1 GiB/  2.1 GiB]   31.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
- [5 files][  2.9 GiB/  2.9 GiB]   20.6 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

0.15381079935942088

In [0]:
embedding_matrix.shape

(199977, 100)

In [0]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           19997700  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
# smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
# smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
# # Set up oversampling with SMOTE (Synthetic Minority Oversampling Technique)
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=12, ratio = 1.0) # Will bring final ratio to 1.0
# x_train_res, y_train_res = sm.fit_sample(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [0]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 768673 samples, validate on 390738 samples
Epoch 1/20
768673/768673 [==============================] - 20s 26us/sample - loss: 0.5190 - acc: 0.7714 - val_loss: 0.5092 - val_acc: 0.7780
Epoch 2/20
768673/768673 [==============================] - 19s 25us/sample - loss: 0.4928 - acc: 0.7848 - val_loss: 0.5098 - val_acc: 0.7776
Epoch 3/20
768673/768673 [==============================] - 20s 25us/sample - loss: 0.4826 - acc: 0.7884 - val_loss: 0.5218 - val_acc: 0.7700
Epoch 4/20
768673/768673 [==============================] - 19s 25us/sample - loss: 0.4753 - acc: 0.7907 - val_loss: 0.5164 - val_acc: 0.7725
Epoch 5/20
768673/768673 [==============================] - 21s 27us/sample - loss: 0.4683 - acc: 0.7931 - val_loss: 0.5253 - val_acc: 0.7797
Epoch 6/20
768673/768673 [==============================] - 20s 25us/sample - loss: 0.4630 - acc: 0.7948 - val_loss: 0.5264 - val_acc: 0.7658
Epoch 7/20
768673/768673 [==============================] - 20s 26us/sample - loss: 0.4575 - acc

In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.8150
Testing Accuracy:  0.7539


In [0]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.5967712633434398


In [0]:
# can't get this to work
plot_model(history)

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [0]:
wrong_preds.op_gender.value_counts()

W    75367
M    20794
Name: op_gender, dtype: int64

In [0]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.08080862368134147

In [0]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2039381,11088773,86569077,W,390505,Loïc,"matt, middle age is over... u know today women are independant...",Kirsten Gillibrand,Congress_Democratic,0.900663,M
2191586,11752563,74450840,W,422861,Tonya,"he has not targeting religion. why do not ya'll stop, read and listen.",Terri Sewell,Congress_Democratic,0.897356,M
1740985,10264122,32570461,M,382056,Leslie,yes. thank you trying to hold her accountable,Tim Kaine,Congress_Democratic,0.498100,W
1156788,6088562,19189340,W,208601,Lori,much appreciated by this american.,Robin Kelly,Congress_Democratic,0.778372,M
2009845,11059237,86569077,W,390114,Barbara,keeping you in my thoughts and prayers!,Kirsten Gillibrand,Congress_Democratic,0.541975,M
2155647,11296627,77234050,W,400525,Andrew,great to see you and brian last night!!,Cathy McMorris Rodgers,Congress_Republican,0.890616,M
2164603,11305583,77234050,W,401053,Anne,yes fiscal responsibility. vote out thr tax and spend liberals come nov.,Cathy McMorris Rodgers,Congress_Republican,0.861994,M
1917154,10966546,86569077,W,389341,Kristen,thank you!,Kirsten Gillibrand,Congress_Democratic,0.667581,M
1900572,10933500,10458808,M,386556,Kris,typical republican tactic they have used wedge issues for years to turn us against each other,Mark Takano,Congress_Democratic,0.426479,W
2166885,11307865,95515852,M,401126,Scott,only reason she will get in cause of the vice pres's vote. total bullshit,Bill Huizenga,Congress_Republican,0.371532,W


In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1912412,10961804,86569077,W,389338,Kristie,the president's job is to keep this country safe. kirsten gillibrand - have you listened to the...,Kirsten Gillibrand,Congress_Democratic,0.999693,M
2050759,11100151,86569077,W,390697,Andrew,hay dave farns----- are you for real go back to your glen beck crap pumping spuddum and stop bre...,Kirsten Gillibrand,Congress_Democratic,0.999638,M
2292114,13627779,4938109,W,529470,Dona,what are we going to do? i usually look at specific issues and where i stand on them and vote a...,Yvette Clarke,Congress_Democratic,0.999526,M
2154850,11295830,77234050,W,400490,Ron,"give up your bush fixation, nathan. it has unhealthy.",Cathy McMorris Rodgers,Congress_Republican,0.999490,M
2007087,11056479,86569077,W,390101,Donna,it will take a decade to rebuild. sandy was worse to ny/nj than andrew was to florida.god be wit...,Kirsten Gillibrand,Congress_Democratic,0.999415,M
2151553,11292533,77234050,W,400340,Nathan,"emm peecchh heeemm!! the job-killing, foriegn born tyranous usurper just swallowed 1 million jo...",Cathy McMorris Rodgers,Congress_Republican,0.999414,M
2127121,11268101,77234050,W,399741,Judith,"did not you hear mr. paul, ""we do not live in a democracy we love in a republic"". :)",Cathy McMorris Rodgers,Congress_Republican,0.999380,M
2140993,11281973,77234050,W,399960,Rob,"probably because they are not conservatives, nathan. neocons outing neocons is kinda entertaini...",Cathy McMorris Rodgers,Congress_Republican,0.999265,M
2103705,11244685,77234050,W,399379,Angel,thanks for not even considering the batwing crazy louie.,Cathy McMorris Rodgers,Congress_Republican,0.999248,M
1921115,10970507,86569077,W,389341,Matt,love love love betsy. john king and arne duncan!!!! buhbye!!!!!,Kirsten Gillibrand,Congress_Democratic,0.999240,M


In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2286672,13470134,36181883,M,517866,Mario,"ola primo steve scalise dos estados unidos,esta na hora de se candidatar como presidente dos es...",Steve Scalise,Congress_Republican,0.000617,W
1058592,5307646,46506154,M,161416,Raimundo,"felizmente todo esta ahora como debe. mis saludos personales para ti, para jennifer, y para cada...",Marco Rubio,Congress_Republican,0.000776,W
589151,4838205,46506154,M,160187,Gladys,https://oiganoticias.com/2016/04/11/senador-de-estados-unidos-busca-evitar-que-terrorista-de-las...,Marco Rubio,Congress_Republican,0.001281,W
1058302,5307356,46506154,M,161416,Segundo,"estimado senador marco rubio mis oraciones, las de mi familia y de toda la comunidad nicaraguens...",Marco Rubio,Congress_Republican,0.001495,W
1900921,10933849,10458808,M,386563,Jahaziel,o q diferencia ve q bonito es saber ingles asi si le entendi hasta parece q hablo en español jul...,Mark Takano,Congress_Democratic,0.001864,W
1419037,7837331,99469231,M,296316,Бойко,"американската субкултура напоследък ни засипва в артефакти на психиатрично творчество, които за ...",John McCain,Congress_Republican,0.002304,W
1581041,7999335,99469231,M,297004,John,free the pow/mia from viet nam now !!!,John McCain,Congress_Republican,0.002445,W
1322094,7740388,99469231,M,296101,Barbara,"thank you , debbie",John McCain,Congress_Republican,0.002568,W
572168,4821222,46506154,M,160159,Larry,if you vote for roberts stabenow i cannot vote for you!,Marco Rubio,Congress_Republican,0.002595,W
1582479,8000773,99469231,M,297010,Matt,minnesota gophers,John McCain,Congress_Republican,0.002598,W
